In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Online Retail.xlsx to Online Retail.xlsx


In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules 
import matplotlib.pyplot as plt


In [ ]:
dt = pd.read_excel("Online Retail.xlsx")
dt.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
dt.columns = dt.columns.str.strip().str.lower().str.replace(" ","_").str.replace('('," ").str.replace(')'," ")
dt.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
dt.shape

(541909, 8)

In [ ]:
dt.invoiceno.nunique()

25900

In [ ]:
dt.customerid.nunique()

4372

In [ ]:
dt.isnull().sum()

invoiceno           0
stockcode           0
description      1454
quantity            0
invoicedate         0
unitprice           0
customerid     135080
country             0
dtype: int64

In [ ]:
dt['invoiceno'] = dt['invoiceno'].astype('str')
dt['invoiceno']=dt[~dt['invoiceno'].str.contains('C')]


In [ ]:
dt.shape

(541909, 8)

In [ ]:
def encode_units(x):
  if x<=0:
    return 0
  if x>=1:
    return 1

In [ ]:
region = "Australia"
#final = pd.DataFrame()
for ctry in list(set(dt['country'])):
  temp = dt.loc[dt['country'] == region].copy()
  basket = temp.groupby(['invoiceno','description'])['quantity'].sum().unstack().reset_index().fillna(0).set_index('invoiceno')
  print(basket.head())
  basket_sets = basket.applymap(encode_units)
  if 'POSTAGE' in basket_sets.columns:
    basket_sets.drop('POSTAGE',inplace=True,axis=1)
  #frequency
  frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
  #print (frequent_itemsets)

  #rules
  rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
  #print(rules.head())
  print("no. of associations = ", rules.shape[0])
  rules['Region']=np.repeat(region,len(rules.index))
  #rules.hist('confidence',grid = False , bins =30)
  #plt.title('confidence')
  #rules.hist('lift',grid = False , bins =30)
  #plt.title('lift')
  print("------------reulst-----------")
  print(rules[ (rules['lift']>=6 ) & (rules['confidence'] >=0.8) ])
  #final = final.append(rules,ignore_index=True)
  break
  
  

description  ALARM CLOCK BAKELIKE GREEN  ...  NOEL GARLAND PAINTED ZINC 
invoiceno                                ...                            
536389                              4.0  ...                         0.0
537676                              0.0  ...                         0.0
539419                              0.0  ...                         0.0
540267                              0.0  ...                         0.0
540280                              0.0  ...                         0.0

[5 rows x 608 columns]
no. of associations =  800
------------reulst-----------
                                           antecedents  ...     Region
0                         (ALARM CLOCK BAKELIKE GREEN)  ...  Australia
1                          (ALARM CLOCK BAKELIKE RED )  ...  Australia
3                               (RED DINER WALL CLOCK)  ...  Australia
5                               (RED DINER WALL CLOCK)  ...  Australia
6                             (RED RETROSPOT UMBRELLA

In [ ]:
rules.to_csv("Association_result_for_online_reatil_data.csv",index=False)
files.download("Association_result_for_online_reatil_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
rules.columns

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'Region'],
      dtype='object')